# Cross Entropy Benchmarking 

Cross entropy benchmarking (XEB) is a method for characterizing the fidelity of a two-qubit gate.
The XEB sequence consists in a series of cycles composed of randomly selected single-qubit gates and a two-qubit gate,
followed by a measurement.
The sequence is repeated many times, and the results are used to calculate the cross entropy between the ideal and measured states.
The cross entropy is a measure of the distance between two probability distributions, and is related to the fidelity of the two-qubit gate.
The cross entropy is calculated as follows:
    1. The ideal state is calculated by applying the sequence of single-qubit gates and the two-qubit gate to the initial state |00>. We use Qiskit to reconstruct the ideal state and for showing the `QuantumCircuit`s that are used.
    2. The measured state is calculated by applying the sequence of single-qubit gates to the initial state |00>, followed by a measurement.
    3. The cross entropy is calculated between the ideal and measured states.
The cross entropy is calculated for a range of sequence depths, and the results are used to compute the fidelity of the two-qubit gate.
In this script, we provide an example of how to run an XEB sequence on the OPX. For the post-processing, we use Qiskit
to reconstruct the ideal state from the randomly generated sequence and to calculate the cross entropy. We then use tools derived from Cirq Experiments library (https://github.com/quantumlib/Cirq/blob/main/cirq-core/cirq/experiments/xeb_fitting.py) to calculate the XEB fidelity of the two-qubit gate.

Note: To run the script, it is necessary to install Qiskit. The reader can refer to the installation guide for Qiskit 1.0 at https://qiskit.org/documentation/install.html or the more recent video: https://youtu.be/dZWz4Gs_BuI?si=EOqyeOhZ05YcBlXA .

Author: Arthur Strauss - Quantum Machines
Created: 16/01/2024 (Last modified: 08/04/2024)

In [ ]:
try:
    from qiskit.circuit import QuantumCircuit
    from qiskit.circuit.library import get_standard_gate_name_mapping as gate_map, UnitaryGate, RYGate
except ImportError:
    !pip install qiskit

In [ ]:
# Imports
from pathlib import Path
import os
import numpy as np
from qm import QuantumMachinesManager
from qm.qua import *
from qualang_tools.addons.variables import assign_variables_to_element
from scipy.stats import stats
from scipy import optimize
from scipy.linalg import sqrtm
import pandas as pd
from typing import Tuple
from config import config
from qm.simulate import SimulationConfig
from matplotlib import pyplot as plt
import seaborn as sns
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import get_standard_gate_name_mapping as gate_map, UnitaryGate, RYGate
from qiskit.quantum_info import Statevector
import json

## Hyperparameters of XEB

Below we define the hyperparameters of the XEB sequence. The hyperparameters are:
- `seqs`: The number of random sequences to run.
- `max_depth`: The maximum depth of the XEB sequence.
- `step`: The step for choosing which depths to run up to `max_depth`.
- `depths`: The array of depths of the XEB sequence to run (can be deduced from `max_depth`and `step` with `np.arange` or can be specified directly as an iterable).
- `n_shots`: The number of shots for each measurement.

We add additional hyperparameters:
- `impose_0_cycle`: Whether to impose the first gate at 0-cycle.
- `save_dir`: The directory where the data will be saved.
- `should_save_data`: Whether to save the data.

In [ ]:
save_dir = ""  # Directory where the data will be saved
should_save_data = True  # Whether to save the data
generate_new_data = True  # Whether to generate new data

# Define hyperparameters of the XEB sequence
seqs = 100  # Number of random sequences to run per depth
max_depth = 3  # Max depth of the XEB sequence
step = 1  # Step for choosing which depths to run up to max_depth
depths = np.arange(1, max_depth + 1, step)  # Create an array of depths to iterate through

n_shots = 101  # Number of averages per sequence
impose_0_cycle = False  # Whether to impose the first gate at 0-cycle
apply_two_qb_gate = False
print("depths array:", depths)

In [ ]:
Path().cwd()

## Defining gates sets for the XEB sequence
Below, we declare the gate sets for the XEB sequence. The gates are declared as Qiskit objects that can be inserted into a `QuantumCircuit`.
Each single qubit gate inserted is assumed to be achievable by either virtual frame rotations (such as $T$ and $S$ gates) or by amplitude modulation of a baseline pulse defined in the configuration (such as $SX$ and $SY$ gates).

The gates are defined as dictionaries, where the keys are integers representing an index that will be used within QUA for real time gate generation, and the values are dictionaries containing the following keys:
- `gate`: The Qiskit gate object.
- `amp_matrix`: The amplitude matrix of the gate. This is used within QUA to fix how the baseline pulse in the configuration will be modulated in real time to engineer the gate.

As a reminder, the matrix for the $SW$ gate (from this paper: https://www.nature.com/articles/s41586-019-1666-5) is defined as:
$$
SW = \sqrt{W} =\sqrt{\frac{X+Y}{\sqrt{2}}}
$$


In [ ]:
# Define the possible gate sets for the XEB sequence (with Qiskit)
SX, T, X, Y, Z, H, CZ, CX, ECR = [gate_map()[gate] for gate in ["sx", "t", "x", "y", "z", "h", "cz", "cx", "ecr"]]
SY = RYGate(np.pi / 2)
W = UnitaryGate((X.to_matrix() + Y.to_matrix()) / np.sqrt(2), label="W")
SW = UnitaryGate(sqrtm(W), label="SW")  # from Supremacy paper

# Define the relevant amplitude matrices for all single-qubit gates (all generated through initial X90 gate)
X90_dict = {"gate": SX, "amp_matrix": np.array([1.0, 0.0, 0.0, 1.0])}
Y90_dict = {"gate": SY, "amp_matrix": np.array([0.0, -1.0, 1.0, 0.0])}
T_dict = {
    "gate": T,
    "amp_matrix": np.array([1.0, 0.0, 0.0, 1]),
}  # No actual need for amp_matrix (done with frame rotation)
SW_dict = {"gate": SW, "amp_matrix": 0.70710678 * np.array([1.0, -1.0, 1.0, 1.0])}

# Possible gate sets
gate_dict1 = {0: X90_dict, 1: Y90_dict, 2: T_dict}  # https://arxiv.org/abs/1608.00263
gate_dict2 = {
    0: X90_dict,
    1: Y90_dict,
    2: SW_dict,
}  # Supremacy gate set (https://www.nature.com/articles/s41586-019-1666-5)
gate_sets = [gate_dict1, gate_dict2]

# Choose the gate set
gate_set_choice = 2  # 1, 2 (as defined above)
gate_dict = gate_sets[gate_set_choice - 1]
random_gates = len(gate_dict)
two_qubit_gate = CZ  # The two-qubit gate to use in the XEB sequence

## Retrieving relevant information from the QUA configuration 
Below, we define all the pieces enabling the definition of the XEB QUA program. This includes the definition of 
- the qubits targeted by the protocol (and their associated elements)
 - their associated readout resonators
 -  the thresholds for the readout.

In [ ]:
qop_ip = ""
qop_port = 80
cluster_name = None
octave_config = None
qmm = QuantumMachinesManager(host=qop_ip, port=qop_port, cluster_name=cluster_name, octave=octave_config)

# Define the qubits and the readout resonators
qubits = [0, 2]  # Fix which qubits indices to use on the chip (quantum elements in the configuration)
n_qubits, dim = len(qubits), 2 ** len(qubits)

qubit_elements = [f"qubit{i}$xy" for i in qubits]  # qubit elements associated to qubits
readout_elements = [f"qubit{i}$rr" for i in qubits]  # readout resonators elements associated to qubits
# CZ operations associated to qubit pairs (tuple is (element, operation) to trigger CZ between the qubits)
CZ_operations = {
    (1, 2): ("qubit2$z", "Cz$flux_pulse_control_qubit1"),
    (0, 2): ("qubit2$z", "Cz$flux_pulse_control_qubit0"),
    (2, 1): ("qubit2$z", "Cz$flux_pulse_control_qubit1"),
    (2, 0): ("qubit2$z", "Cz$flux_pulse_control_qubit0"),
    (2, 3): ("qubit3$z", "Cz$flux_pulse_control_qubit2"),
    (3, 2): ("qubit3$z", "Cz$flux_pulse_control_qubit2"),
}

# Load additional information for enabling readout

# Define the threshold for the readout
ge_thresholds, rus_thresholds = [0.5] * n_qubits, [
    0.5
] * n_qubits  # Thresholds for the readout and repeat-until-success (RUS) protocols for active reset
res_deplete_times = [50] * n_qubits  # Time for the resonator to deplete

simulate = False

In [ ]:
if n_qubits < 2:
    apply_two_qb_gate = False

# For Data Saving:
qubits_involved = "_".join([str(x) for x in qubits])
comment = f"s({seqs})_d({max_depth})_g({gate_set_choice})_cz({int(apply_two_qb_gate)})_0cyc({int(impose_0_cycle)})"
filename = f"XEB_q{qubits_involved}_{comment}"

## Defining some useful QUA macros

Before declaring the program, we insert some useful QUA macros that will be used in the XEB program. These macros include:
- `assign_amplitude_matrix`: Assigns the amplitude matrix arguments for a given gate index. This assignment is done in real time through the use of a QUA switch-case statement.
- `qua_declaration`: Declares the necessary QUA variables for storing the readout results (in-phase and quadrature components).
- `readout`: Performs the readout of a qubit. The readout is done by playing a readout pulse on the readout resonator element associated with the qubit. The in-phase and quadrature components of the readout signal are saved in QUA streams.
- `cz_gate`: Performs the CZ gate between two qubits. The CZ gate is performed by playing the CZ operation on the element associated with the CZ gate.
- `play_T_gate_set`: Defines the play command for the T gate set. The T gate set consists of the $SX$, $SY$, and $T$ gates. $SY$ is played from the baseline $SX$ gate, modulated with a specific amplitude matrix. For the $T$ gate, a frame rotation is applied.
- `play_SW_gate_set`: Defines the play command for the SW gate set. The SW gate set consists of the $SX$, $SY$, and $SW$ gates. $SY$ is played as its own calibrated gate (could actually be done in a similar way to the method associated to $T$ gate set, we show both cases for completeness)/ For the $SW$ gate, the $SX$ gate is played with a specific amplitude matrix enabling a rotation around the $X+Y$ axis.

In [ ]:
def assign_amplitude_matrix(gate, a):
    """
    QUA Macro for assigning the amplitude matrix arguments for a given gate index.
    :param gate: Gate index
    :param a: Amplitude matrix arguments
    """
    with switch_(gate):
        for i in range(random_gates):
            with case_(i):
                for j in range(4):
                    assign(a[j], gate_dict[i]["amp_matrix"][j])


def qua_declaration(n_qubits):
    """
    Macro to declare the necessary QUA variables

    :param n_qubits: Number of qubits used in this experiment
    :return:
    """
    I, Q = [[declare(fixed) for _ in range(n_qubits)] for _ in range(2)]
    I_st, Q_st = [[declare_stream() for _ in range(n_qubits)] for _ in range(2)]
    # Workaround to manually assign the results variables to the readout elements
    for i in range(n_qubits):
        assign_variables_to_element(readout_elements[i], I[i], Q[i])
    return I, I_st, Q, Q_st


# QUA Program


def align_qubit(qubit: int):
    base_qubit = f"qubit{qubit}$"
    elements = [base_qubit + el for el in ["xy", "rr", "z"]]
    align(*elements)


def readout(I, I_st, Q, Q_st, qubit: int, amplitude=1.0):
    """
    Macro for qubit readout
    :param I: Variable for the in-phase component of the readout signal
    :param I_st: Stream for the in-phase component of the readout signal
    :param Q: Variable for the quadrature component of the readout signal
    :param Q_st: Stream for the quadrature component of the readout signal
    :param qubit: Qubit index
    :param amplitude: Amplitude of the readout signal
    """
    rr = f"qubit{qubit}$rr"
    integration_weights = [f"iw{i}" for i in range(1, 4)]

    amp_var = None
    if isinstance(amplitude, tuple):
        assert len(amplitude) == 4, "Amplitude matrix not complete"
        amp_var = amp(*amplitude)
    elif amplitude != 1.0:
        amp_var = amp(amplitude)
    if amp_var is not None:
        pulse = "readout" * amp_var
    else:
        pulse = "readout"

    measure(
        pulse,
        rr,
        None,
        dual_demod.full(integration_weights[0], "out1", integration_weights[1], "out2", I),
        dual_demod.full(integration_weights[2], "out1", integration_weights[0], "out2", Q),
    )

    if I_st is not None:
        save(I, I_st)
    if Q_st is not None:
        save(Q, Q_st)


def cz_gate(control, target):
    """
    QUA Macro for the CZ gate
    :param control: Control qubit index
    :param target: Target qubit index
    """
    cz_el, cz_op = CZ_operations[(control, target)]
    play(cz_op, cz_el)


def play_T_gate_set(g, a, qubit_el):
    """
    QUA Macro for the T gate set
    The two first cases will focus on the X90 and Y90 gates, while the last case will focus on the T gate.
    :param g: Gate index
    :param a: Amplitude matrix
    :param qubit_el: Qubit element
    """
    with switch_(g, unsafe=True):
        for j in range(2):  # X90, Y90
            with case_(j):
                play("sx" * amp(*a), qubit_el)
        with case_(2):  # T gate
            frame_rotation(np.pi / 4, qubit_el)


def play_SW_gate_set(g, a, qubit_el):
    """
    QUA Macro for the SW gate set
    The two first cases will focus on the X90 and Y90 gates, while the last case will focus on the SW gate.
    :param g: Gate index
    :param a: Amplitude matrix
    :param qubit_el: Qubit element
    """
    assert len(gate_dict) == 3, "Provided gate set does not contain 3 sq gates"
    with switch_(g, unsafe=True):
        for j, gate in enumerate(["sx", "sy"]):  # X90, Y90, SW
            with case_(j):
                play(gate, qubit_el)
        with case_(2):
            play("sx" * amp(*a), qubit_el)


def play_random_sq_gate(g, a, qubit_el, gate_dict):
    """
    QUA Macro for playing a random single-qubit gate
    :param g: Gate index
    :param a: Amplitude matrix
    :param qubit_el: Qubit element
    :param gate_dict: Dictionary of gates
    """
    if T_dict in gate_dict.values():  # T gate involves frame rotation
        play_T_gate_set(g, a, qubit_el)
    elif SW_dict in gate_dict.values():
        play_SW_gate_set(g, a, qubit_el)
    else:
        play("sx" * amp(*a), qubit_el)


def active_reset(qubit, qubit_el, I, Q, res_deplete_time, rus_threshold, ge_threshold, state):
    """
    QUA Macro for active reset
    :param qubit: Qubit index
    :param qubit_el: Qubit element
    :param I: In-phase component of the readout signal
    :param Q: Quadrature component of the readout signal
    :param res_deplete_time: Time for the resonator to deplete
    :param rus_threshold: Threshold for the resonator to exit the reset
    :param ge_threshold: Threshold for the qubit to be in the ground state
    :param state: State of the qubit
    """
    wait(res_deplete_time // 4, qubit_el)
    play("x", qubit_el, condition=state)

    align()
    # Active reset
    with while_(I > rus_threshold):
        align_qubit(qubit)
        readout(I, None, Q, None, qubit)
        assign(state, I > ge_threshold)
        wait(res_deplete_time // 4, qubit_el)
        play("x", qubit_el, condition=state)
        align_qubit(qubit)


def binary(n, length):
    """
    Convert an integer to a binary string of a given length
    :param n: Integer to convert
    :param length: Length of the binary string
    :return: Binary string
    """
    return bin(n)[2:].zfill(length)


def cross_entropy(p, q, epsilon=1e-15):
    """
    Calculate cross entropy between two probability distributions.

    Parameters:
    - p: numpy array, the true probability distribution
    - q: numpy array, the predicted probability distribution
    - epsilon: small value to avoid taking the logarithm of zero

    Returns:
    - Cross entropy between p and q
    """
    q = np.maximum(q, epsilon)  # Avoid taking the logarithm of zero
    x_entropy = -np.sum(p * np.log(q))
    return x_entropy

## Defining the XEB QUA program

We now have all the necessary elements to define the XEB QUA program. The program consists of the following steps:
1. Declare the QUA variables. In this stage, the two particular variables to be noted are the variables `g` and `a`. 
   - The variable `g` is a list whose elements are QUA arrays of size `max_depth`. Each element of the list corresponds to a qubit gate sequence, and each element within this gate sequence is an integer index to be sampled randomly referring to the corresponding gate in the chosen `gate_dict` defined earlier.
   - The variable `a` is a list of lists of QUA arrays of size `max_depth`. The outer dimension corresponds to the qubit index, and the inner dimension corresponds to the amplitude matrix of the gate at a given depth. To facilitate the looping over the depth using a QUA for loop, the depth is set in the most inner dimension.

For each random sequence and for each depth of the selected `depths` array, the program does the following:
2. Generate the random sequences of single-qubit gates. The random sequences are generated for each qubit, and the gate indices are stored in the `g` variable. Note that we save all the sampled indices in the stream-processing in order to reconstruct in the post-processing the circuit that was run and deduce from it the ideal (noise-free) quantum state.
3. Run the XEB sequence. The XEB sequence consists of the following steps:
       - Reset the qubits to their ground states.
       - Apply successive layers composed of two single-qubit gates and one two-qubit gate for each depth of the XEB sequence.
       - Measure the state of the qubits.
       - Convert the measured state to a bitstring and save the associated counts in stream-processing.


In [ ]:
with program() as xeb:
    # Declare QUA variables
    I, I_st, Q, Q_st = qua_declaration(n_qubits=n_qubits)
    d, d_, n, s, tot_state_ = [declare(int) for _ in range(5)]  # Max_depth index, growing depth index, shot index
    g = [declare(int, size=depths[-1]) for _ in range(n_qubits)]  # Gate indices list for both qubits
    a = [
        [declare(fixed, size=depths[-1]) for _ in range(4)] for _ in range(n_qubits)
    ]  # Amplitude matrices for both qubits (for all depths)
    counts = [declare(int, value=0) for _ in range(dim)]  # Counts for all possible bitstrings (00, 01, 10, 11)
    state = [declare(bool) for _ in range(n_qubits)]  # State of the qubits
    # Declare streams
    counts_st = [
        declare_stream() for _ in range(dim)
    ]  # Stream for the counts of all possible bitstrings (00, 01, 10, 11)
    state_st = [declare_stream() for _ in range(n_qubits)]  # Stream for the individual state of the qubits
    g_st = [
        declare_stream() for _ in range(n_qubits)
    ]  # Stream for the gate indices (enabling circuit reconstruction in post-processing)

    # Setting the seed for reproducibility
    r = Random()
    r.set_seed(12321)

    # If we are simulating, we update the frequency of the qubits to 0 to visualize the sequence
    if simulate:
        a_st = [[declare_stream() for _ in range(4)] for _ in range(n_qubits)]
        for qubit in qubit_elements:
            update_frequency(qubit, 0)

    # Generate the random sequences
    with for_(s, 0, s < seqs, s + 1):
        with for_each_(d, depths):
            # NOTE: randomizing is done for each growing-depths and sequence (some other strategies could be used)
            for q in range(n_qubits):
                first_gate = random_gates - 1 if impose_0_cycle else random_gates
                assign(g[q][0], r.rand_int(first_gate))
                save(g[q][0], g_st[q])
                with for_(d_, 1, d_ < d, d_ + 1):
                    assign(g[q][d_], r.rand_int(random_gates))
                    with while_(g[q][d_] == g[q][d_ - 1]):  # Make sure the same gate is not applied twice in a row
                        assign(g[q][d_], r.rand_int(random_gates))
                    # Map the sequence indices into amplitude matrix arguments (each index corresponds to a random gate)
                    assign_amplitude_matrix(g[q][d_], [a[q][i][d_] for i in range(4)])
                    save(g[q][d_], g_st[q])

                    if simulate:
                        for amp_matrix_element in range(4):
                            save(a[q][amp_matrix_element][d_], a_st[q][amp_matrix_element])

            # Run the XEB sequence
            with for_(n, 0, n < n_shots, n + 1):
                # Reset the qubits to their ground states (here simple wait but could be an active reset macro)
                if simulate:
                    wait(25, *qubit_elements)
                else:
                    # wait(3 * thermalization_time, *qubit_elements)
                    pass  # active reset below

                # NOTE: imposing first gate at 0-cycle:
                if impose_0_cycle:
                    for q in qubit_elements:
                        play("sx" * amp(*SW_dict["amp_matrix"]), q)  # Apply the SW gate

                # Play all cycles generated for sequence s of depth d
                with for_(d_, 0, d_ < d, d_ + 1):
                    for q, (qubit, qubit_el) in enumerate(
                        zip(qubits, qubit_elements)
                    ):  # Play single qubit gates on both qubits
                        play_random_sq_gate(g[q][d_], [a[q][i][d_] for i in range(4)], qubit_el, gate_dict)

                    # Insert your two-qubit gate macro here
                    if apply_two_qb_gate:
                        align_qubit(qubit)
                        cz_gate(qubits[0], qubits[1])
                        align_qubit(qubit)

                # Measure the state (insert your readout macro here)
                for q_idx, (qubit, qubit_el) in enumerate(zip(qubits, qubit_elements)):
                    readout(I[q_idx], I_st[q_idx], Q[q_idx], Q_st[q_idx], qubit)
                    # State Estimation: returned as an integer, to be later converted to bitstrings
                    assign(state[q_idx], I[q_idx] > ge_thresholds[q_idx])
                    save(state[q_idx], state_st[q_idx])
                    assign(tot_state_, tot_state_ + 2**q_idx * Cast.to_int(state[q_idx]))

                    active_reset(
                        qubit,
                        qubit_el,
                        I[q_idx],
                        Q[q_idx],
                        res_deplete_times[q_idx],
                        rus_thresholds[q_idx],
                        ge_thresholds[q_idx],
                        state[q_idx],
                    )

                with switch_(tot_state_):
                    for i in range(dim):  # Bitstring conversion
                        with case_(i):
                            assign(counts[i], counts[i] + 1)  # counts for 00, 01, 10 and 11
                assign(tot_state_, 0)  # Resetting the state
            for i in range(dim):  # Resetting Bitstring collection
                save(counts[i], counts_st[i])
                assign(counts[i], 0)

    # Save the results
    with stream_processing():
        for q in range(n_qubits):
            g_st[q].save_all(f"g{q}")
            I_st[q].buffer(n_shots).map(FUNCTIONS.average()).buffer(len(depths)).save_all(f"I{q}")
            Q_st[q].buffer(n_shots).map(FUNCTIONS.average()).buffer(len(depths)).save_all(f"Q{q}")
            state_st[q].boolean_to_int().buffer(n_shots).map(FUNCTIONS.average()).buffer(len(depths)).save_all(
                f"state{q}"
            )
        for i in range(dim):
            string = "s" + binary(i, n_qubits)
            counts_st[i].buffer(len(depths)).save_all(string)

        if simulate:
            for q in range(n_qubits):
                for d_ in range(4):
                    a_st[q][d_].save_all(f"a{q + 1}_{binary(d_, 2)}")

## Running the XEB program on the OPX

In [ ]:
if simulate:
    job = qmm.simulate(config, xeb, SimulationConfig(15000))
    job.get_simulated_samples().con1.plot()
    job.get_simulated_samples().con2.plot()
    plt.show()
elif generate_new_data:
    qm = qmm.open_qm(config)
    job = qm.execute(xeb)
else:
    job = None

if job is not None:
    job.result_handles.wait_for_all_values()
    result = job.result_handles

## Retrieving data from stream-processing

In [ ]:
# Retrieve the data from the streams
if generate_new_data or simulate:
    g = [result.get(f"g{i}").fetch_all()["value"] for i in range(n_qubits)]

    # Rebuild gate sequences generated from QUA:
    sq_indices = []
    idx = 0
    for s in range(seqs):
        sq_indices.append([])
        for i, d in enumerate(depths):
            sq_indices[s].append(np.zeros((n_qubits, d), dtype=int))
            for d_ in range(d):
                for q in range(n_qubits):
                    sq_indices[s][i][q][d_] = g[q][idx]
                idx += 1
    if simulate:
        a = {
            f"a{q + 1}_{binary(i, 2)}": result.get(f"a{q + 1}_{binary(i, 2)}").fetch_all()["value"]
            for q in range(n_qubits)
            for i in range(4)
        }
    else:
        quadratures = {f"I{i}": result.get(f"I{i}").fetch_all()["value"] for i in range(n_qubits)}
        quadratures.update({f"Q{i}": result.get(f"Q{i}").fetch_all()["value"] for i in range(n_qubits)})

        state = {f"state{i}": result.get(f"state{i}").fetch_all()["value"] for i in range(n_qubits)}
        counts = {binary(i, n_qubits): result.get(f"s{binary(i, n_qubits)}").fetch_all()["value"] for i in range(dim)}

        qm.close()

## Saving the data

In [ ]:
# Data saving:
comment = "test"
filename = f"XEB_q{'-'.join([str(x) for x in qubits])}_{comment}"

if should_save_data and not simulate and generate_new_data:
    np.savez(
        save_dir / filename,
        sq_indices=np.array(sq_indices),
        a=a,
        quadratures=quadratures,
        state=state,
        counts=counts,
        gate_set_choice=gate_set_choice,
        seqs=seqs,
        max_depth=max_depth,
        step=step,
        n_shots=n_shots,
        qubits=qubits,
    )
    print("Data saved as %s.npz" % filename)

# Loading existing data

In [ ]:
import fnmatch
import os

flist = fnmatch.filter(os.listdir(save_dir), "XEB*")
keyword = ""
flist = list(filter(lambda x: keyword in x, flist))
print("Saved data with keyword '%s':\n" % keyword)
for i, f in enumerate(flist):
    print("%s. %s" % (i, f))

In [ ]:
# Loading selected data:
if not generate_new_data:
    filename = flist[1]
    npz_file = np.load(save_dir / f"{filename}", allow_pickle=True)

    # Create an empty dictionary to store variables
    variables = {}

    # Iterate through keys and store variables in the dictionary
    for key in npz_file.files:
        variables[key] = npz_file[key]

    # Close the npz file after loading the data
    npz_file.close()

    # loading variables from file:
    sq_indices = variables["sq_indices"]
    a = variables["a"]
    quadratures = variables["quadratures"]
    state = variables["state"].item()
    # counts = variables['counts'][()]
    counts = variables["counts"].item()
    gate_set_choice = variables["gate_set_choice"]
    seqs = variables["seqs"]
    max_depth = variables["max_depth"]
    step = variables["step"]
    depths = np.arange(1, max_depth + 1, step)
    avgs = variables["avgs"]
    qubits = variables["qubits"]
    apply_cz = variables["apply_cz"]
    impose_0_cycle = variables["impose_0_cycle"]

    print("qubits = %s" % qubits)
    print("seqs = %s" % seqs)
    print("max_depth = %s" % max_depth)
    print("step = %s" % step)
    print("avgs = %s" % avgs)
    print("apply_cz = %s" % apply_cz)
    print("impose_0_cycle = %s" % impose_0_cycle)
    print("gate_set_choice = %s" % gate_set_choice)

    print(counts)

## Post-processing the data

Below, we handle the post-processing of the data. We first define the cross-entropy function, which calculates the cross-entropy between two probability distributions. We then reconstruct the ideal state from the randomly generated sequence and calculate the cross entropy. Finally, we calculate the XEB fidelity of the two-qubit gate using tools derived from the Cirq Experiments library. Note that we are using two different estimators for the fidelity, one is the usual log-XEB estimator, and the other is the linear estimator (used in the Cirq Experiments library).
We also identify singularities and outliers in the data and calculate the percentage of singularities and outliers.


In [ ]:
if simulate:
    raise ValueError("Cannot post-process simulated data")

records = []
incoherent_distribution = np.ones(dim) / (dim)
expected_probs = np.zeros((seqs, len(depths), dim))
measured_probs = np.zeros((seqs, len(depths), dim))
log_fidelities = np.zeros((seqs, len(depths)))

# Reconstruct every Circuit from the previously rebuilt gate sequences: (using Qiskit)
circuits_list = []
singularity = []
outlier = []
for s in range(seqs):
    circuits_list.append([])
    for d_, d in enumerate(depths):
        qc = QuantumCircuit(n_qubits)
        # NOTE: imposing first gate at 0-cycle:
        gate_cycle_0 = SW  # XY90, H
        if impose_0_cycle:
            for q in range(n_qubits):
                qc.append(gate_cycle_0, [q])

        for k in range(d):  # Apply the sequence of gates
            sq_gates = [gate_dict[sq_indices[s][d_][q][k]]["gate"] for q in range(n_qubits)]
            for q in range(n_qubits):
                qc.append(sq_gates[q], [q])  # Apply the single-qubit gates
            if apply_two_qb_gate:
                qc.append(CZ, [0, 1])  # Apply the two-qubit gate

        circuits_list[s].append(qc)  # Save the circuit
        expected_probs[s, d_] = np.round(Statevector(qc).probabilities(), 5)  # [1, 0]
        measured_probs[s, d_] = np.array([counts[bin(i)[2:].zfill(n_qubits)][s][d_] for i in range(dim)]) / n_shots

        xe_incoherent = cross_entropy(incoherent_distribution, expected_probs[s, d_])
        xe_measured = cross_entropy(measured_probs[s, d_], expected_probs[s, d_])
        xe_expected = cross_entropy(expected_probs[s, d_], expected_probs[s, d_])

        f_log_xeb = (xe_incoherent - xe_measured) / (xe_incoherent - xe_expected)
        if np.isinf(f_log_xeb) or np.isnan(f_log_xeb):
            singularity.append((s, d_))
            log_fidelities[s, d_] = np.nan  # Set all singularities to NaN
        elif f_log_xeb < 0 or f_log_xeb > 1:
            outlier.append((s, d_))
            log_fidelities[s, d_] = np.nan  # Set all outliers to NaN
        else:
            log_fidelities[s, d_] = f_log_xeb

            records += [
                {
                    "sequence": s,
                    "depth": depths[d_],
                    "pure_probs": expected_probs[s, d_],
                    "sampled_probs": measured_probs[s, d_],
                    "circuit": circuits_list[s][d_],
                }
            ]

print(f"singularities (for log XEB cost function): {singularity}")
print(f"overall singularities (for log XEB cost function): {len(singularity) / seqs / len(depths) * 100}%")
print(f"outliers (for log XEB cost function): {outlier}")
print(f"overall outliers (for log XEB cost function): {len(outlier) / seqs / len(depths) * 100}%")

In [ ]:
for record in records:
    e_u = np.sum(record["pure_probs"] ** 2)
    u_u = np.sum(record["pure_probs"]) / dim
    m_u = np.sum(record["pure_probs"] * record["sampled_probs"])
    record.update(e_u=e_u, u_u=u_u, m_u=m_u)

df = pd.DataFrame(records)
df["y"] = df["m_u"] - df["u_u"]
df["x"] = df["e_u"] - df["u_u"]

df["numerator"] = df["x"] * df["y"]
df["denominator"] = df["x"] ** 2

## Plotting the state heatmap
Below, we plot the heatmap of the measured and expected states for each sequence and depth. The measured states are represented by the left column, while the expected states are represented by the right column. The rows represent the different depths, and the columns represent the different random sequences. This is a good way to check the matching between the expected and measured states for each sequence and depth, as well as the decay towards the completely mixed state at large depths.

In [ ]:
def create_subplot(data, subplot_number, title):
    print(title)
    print("data: %s" % data)
    print(subplot_number)
    plt.subplot(subplot_number)
    # plt.pcolor(depths, range(seqs), np.abs(data), vmin=0., vmax=1.)
    plt.pcolor(depths, range(seqs), np.abs(data))
    ax = plt.gca()
    ax.set_title(title)
    if subplot_number > 244:
        ax.set_xlabel("Circuit depth")
    ax.set_ylabel("Sequences")
    ax.set_xticks(depths)
    ax.set_yticks(np.arange(1, seqs + 1))
    plt.colorbar()


titles, data = [], []
for i in range(dim):
    titles.append(f"<{bin(i)[2:].zfill(n_qubits)}> Measured")
    titles.append(f"<{bin(i)[2:].zfill(n_qubits)}> Expected")
    data.append(measured_probs[:, :, i])
    data.append(expected_probs[:, :, i])

plot_number = [241, 242, 243, 244, 245, 246, 247, 248]

k = 0
for title, d, n in zip(titles, data, plot_number):

    # if k%2 == 0:
    # plt.figure()
    # if k==0:
    qubits_involved = ""
    for q in qubits:
        qubits_involved += f"q{q}-"
    plt.suptitle(
        f"XEB for "
        + qubits_involved
        + f", shots: {n_shots}, sequences: {seqs}, max-depth: {max_depth}, two_qubit_gate: {two_qubit_gate.name}"
    )
    create_subplot(d, n, title)
    plt.subplots_adjust(wspace=0.1, hspace=0.7)
    k += 1

    plt.tight_layout()
    plt.ion()

In [ ]:
colors = sns.cubehelix_palette(n_colors=len(depths))
colors = {k: colors[i] for i, k in enumerate(depths)}

_lines = []


def per_cycle_depth(df):
    fid_lsq = df["numerator"].sum() / df["denominator"].sum()

    cycle_depth = df.name
    xx = np.linspace(0, df["x"].max())
    (l,) = plt.plot(xx, fid_lsq * xx, color=colors[cycle_depth])
    plt.scatter(df["x"], df["y"], color=colors[cycle_depth])

    global _lines
    _lines += [l]  # for legend
    return pd.Series({"fidelity": fid_lsq})


linear_fidelities = df.groupby("depth").apply(per_cycle_depth).reset_index()
plt.xlabel(r"$e_U - u_U$", fontsize=18)
plt.ylabel(r"$m_U - u_U$", fontsize=18)
_lines = np.asarray(_lines)
plt.legend(_lines[[0, -1]], depths[[0, -1]], loc="best", title="Cycle depth")
plt.title("q-%s: Fxeb_linear = %s" % (qubits, [linear_fidelities["fidelity"][x] for x in [0, 1]]))
plt.tight_layout()

## Plotting the XEB fidelity

The XEB fidelity is computed through two different post-processing methods:
- Derivation of the linear XEB (as done in the following notebook: https://quantumai.google/cirq/noise/qcvv/xeb_theory#estimating_fidelity)
- Derivation of the log-entropy XEB (as done in the following paper: https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.125.120504)
In both cases, we perform a exponential fit to the data to extract the layer fidelity.

We explicitly take functions from the Cirq experiments library to fit the data. The functions are taken from the following Cirq code:
https://github.com/quantumlib/Cirq/blob/main/cirq-core/cirq/experiments/xeb_fitting.py

For more information about Cirq and Google Quantum AI, please visit this website: https://quantumai.google/cirq

In [ ]:
# Define Cirq functions for fitting (redefined here for avoiding additional dependencies)
# Those functions are slightly adapted to deal with possible singularities and outliers in the data
def exponential_decay(cycle_depths: np.ndarray, a: float, layer_fid: float) -> np.ndarray:
    """An exponential decay for fitting.

    This computes `a * layer_fid**cycle_depths`

    Args:
        cycle_depths: The various depths at which fidelity was estimated. This is the independent
            variable in the exponential function.
        a: A scale parameter in the exponential function.
        layer_fid: The base of the exponent in the exponential function.
    """
    return a * layer_fid**cycle_depths


def _fit_exponential_decay(cycle_depths: np.ndarray, fidelities: np.ndarray) -> Tuple[float, float, float, float]:
    """Fit an exponential model fidelity = a * layer_fid**x using nonlinear least squares.

    This uses `exponential_decay` as the function to fit with parameters `a` and `layer_fid`.
    This function is taken from the following Cirq code: https://github.com/quantumlib/Cirq/blob/main/cirq-core/cirq/experiments/xeb_fitting.py

    Args:
        cycle_depths: The various depths at which fidelity was estimated. Each element is `x`
            in the fit expression.
        fidelities: The estimated fidelities for each cycle depth. Each element is `fidelity`
            in the fit expression.

    Returns:
        a: The first fit parameter that scales the exponential function, perhaps accounting for
            state prep and measurement (SPAM) error.
        layer_fid: The second fit parameters which serves as the base of the exponential.
        a_std: The standard deviation of the `a` parameter estimate.
        layer_fid_std: The standard deviation of the `layer_fid` parameter estimate.
    """
    cycle_depths = np.asarray(cycle_depths)
    fidelities = np.asarray(fidelities)
    mask = (fidelities > 0) & (fidelities < 1)
    print()
    masked_cycle_depths = cycle_depths[mask]
    masked_fidelities = fidelities[mask]

    log_fidelities = np.log(masked_fidelities)

    slope, intercept, _, _, _ = stats.linregress(masked_cycle_depths, log_fidelities)
    layer_fid_0 = np.clip(np.exp(slope), 0, 1)
    a_0 = np.clip(np.exp(intercept), 0, 1)

    try:
        (a, layer_fid), pcov = optimize.curve_fit(
            exponential_decay,
            masked_cycle_depths,
            masked_fidelities,
            p0=(a_0, layer_fid_0),
            bounds=((0, 0), (1, 1)),
            nan_policy="omit",
        )
    except ValueError:  # pragma: no cover
        return 0, 0, np.inf, np.inf

    a_std, layer_fid_std = np.sqrt(np.diag(pcov))
    return a, layer_fid, a_std, layer_fid_std

In [ ]:
fit_linear = True
fit_log_entropy = True
xx = np.linspace(0, linear_fidelities["depth"].max())

try:  # Fit the data for the linear XEB
    if fit_linear:
        a_lin, layer_fid_lin, a_std_lin, layer_fid_std_lin = _fit_exponential_decay(
            linear_fidelities["depth"], linear_fidelities["fidelity"]
        )
        plt.plot(
            xx,
            exponential_decay(xx, a_lin, layer_fid_lin),
            label="Fit (Google processing), layer_fidelity={:.1f}%".format(layer_fid_lin * 100),
            color="red",
        )
except Exception as e:
    print("Fit for Google processing data failed")

Fxeb = np.nanmean(log_fidelities, axis=0)

try:  # Fit the data for the log-entropy XEB
    if fit_log_entropy:
        a_log, layer_fid_log, a_std_log, layer_fid_std_log = _fit_exponential_decay(depths, Fxeb)
        plt.plot(
            xx,
            exponential_decay(xx, a_log, layer_fid_log),
            label="Fit (Log-entropy processing), layer_fidelity={:.1f}%".format(layer_fid_log * 100),
            color="green",
        )
except Exception as e:
    print("Fit for Log-entropy processing data failed")

mask_lin = (linear_fidelities["fidelity"] > 0) & (linear_fidelities["fidelity"] < 1)
masked_linear_depths = linear_fidelities["depth"][mask_lin]
masked_linear_fids = linear_fidelities["fidelity"][mask_lin]
if fit_linear:
    plt.scatter(masked_linear_depths, masked_linear_fids, color="red", label="Google processing")

mask_log = (Fxeb > 0) & (Fxeb < 1)
if fit_log_entropy:
    plt.scatter(depths[mask_log], Fxeb[mask_log], marker="o", color="green", label="Log-entropy processing")

plt.ylabel("Circuit fidelity", fontsize=20)
plt.xlabel("Cycle Depth $d$", fontsize=20)
plt.title("XEB Fidelity")
plt.legend(loc="best")
plt.tight_layout()
plt.show()